In [1]:
# Importing basic data science libraries
import numpy as np       # For numerical computing
import pandas as pd      # For data manipulation and analysis
import matplotlib.pyplot as plt  # For data visualization
import seaborn as sns    # For enhanced data visualization
import sklearn           # For machine learning algorithms and tools
from matplotlib.ticker import FormatStrFormatter
import pickle
import random
import plotly.graph_objects as go
from itertools import combinations
from IPython.display import Image, display
import plotly.graph_objects as go
import io

In [2]:
# Load the solution dictionary from a file
with open('solution.pkl', 'rb') as f:
    loaded_solution = pickle.load(f)


In [3]:
n_CR = loaded_solution.critical_regions

In [4]:
CRs= []
for i in n_CR:
   CRs.append((i.E,(i.f).reshape(-1,)))

In [5]:
import numpy as np
from scipy.optimize import linprog
from itertools import combinations

def find_vertices(A, b, fixed_vars, fixed_values): #, free_var_ranges
    n = A.shape[1]
    free_vars = list(set(range(n)) - set(fixed_vars))

    if len(free_vars) != 3:
        raise ValueError("Exactly 3 free variables are required.")
    
    # Adjust b to account for the fixed variables
    b_new = b - A[:, fixed_vars] @ fixed_values
    A_free = A[:, free_vars]

    vertices = []

    # Iterate through combinations of constraints to find vertices
    for comb in combinations(range(len(b)), 3):
        A_eq = A_free[list(comb), :]
        b_eq = b_new[list(comb)]

        # Solve A_eq * x = b_eq
        try:
            vertex = np.linalg.solve(A_eq, b_eq)

            # # Check if the vertex is within bounds
            # within_bounds = all(free_var_ranges[i][0] <= vertex[i] <= free_var_ranges[i][1] for i in range(3))
            # if not within_bounds:
            #     continue

            # Check if the vertex satisfies all inequalities
            if np.all(A_free @ vertex <= b_new + 1e-9):
                vertices.append(vertex)
        except np.linalg.LinAlgError:
            continue

    vertices = np.array(vertices)
    return np.array(vertices)

# # gamma, sigma, phi, A, D
# fixed_vars = [3,4,5,6,7,8,9,10,11]  # Indices of the variables to fix
# fixed_values = [67.95,1.03,1.55,1.29,12.4,29.9,18.7,62.2,93.4]# Values to fix the variables to
# # free_var_ranges = [(-0.15, 0.15),(225,450),(0.01*450, 0.4*450)]  # Ranges for the 3 free variables (0.1*120, 0.99*120), (0.01*450, 0.4*450)

# CR_index = []
# poly = []
# for i in range(len(CRs)):
#     vertices = find_vertices(CRs[i][0], CRs[i][1], fixed_vars, fixed_values)
#     if np.shape(vertices) == np.shape(np.array([])):
#         pass
#     else:
#         poly.append(vertices)
#         CR_index.append(i)


In [6]:
import numpy as np
import random
import plotly.graph_objects as go
from IPython.display import Image, display

def plot_polytope_from_fixed_values(fixed_values, CRs, extra_points, width=2000, height=1600, axis_limit=None, save_path=None):
    fixed_vars = [3, 4, 5, 6, 7, 8, 9, 10, 11]
    CR_index = []
    poly = []

    for i, (A, b) in enumerate(CRs):
        verts = find_vertices(A, b, fixed_vars, fixed_values)
        if verts.size > 0:
            poly.append(verts)
            CR_index.append(i)

    random.seed(125)
    fig = go.Figure()

    for i, vertices in enumerate(poly):
        x, y, z = vertices.T
        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            opacity=0.3,
            color=f"rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})",
            alphahull=0,
            showscale=False
        ))

        centroid = np.mean(vertices, axis=0)
        fig.add_trace(go.Scatter3d(
            x=[centroid[0]],
            y=[centroid[1]],
            z=[centroid[2]],
            mode='text',
            text=[f"CR {CR_index[i]+1}"],
            textposition='middle center',
            textfont=dict(color='black', size=22),
            showlegend=False
        ))

    extra_array = np.array(extra_points)

    if len(extra_array) > 1:
        fig.add_trace(go.Scatter3d(
            x=extra_array[:, 0],
            y=extra_array[:, 1],
            z=extra_array[:, 2],
            mode='lines',
            line=dict(color='blue', width=4),
            name='Trajectory'
        ))

        fig.add_trace(go.Scatter3d(
            x=extra_array[1:-1, 0],
            y=extra_array[1:-1, 1],
            z=extra_array[1:-1, 2],
            mode='markers',
            marker=dict(size=8, color='black'),
            name='Intermediate Points'
        ))

        fig.add_trace(go.Scatter3d(
            x=[extra_array[0, 0]],
            y=[extra_array[0, 1]],
            z=[extra_array[0, 2]],
            mode='markers+text',
            marker=dict(size=12, color='green', symbol='circle'),
            text=["Start"],
            textposition='middle left',
            textfont=dict(color='black', size=18),
            name='Start Point'
        ))

        fig.add_trace(go.Scatter3d(
            x=[extra_array[-1, 0]],
            y=[extra_array[-1, 1]],
            z=[extra_array[-1, 2]],
            mode='markers+text',
            marker=dict(size=12, color='red', symbol='diamond'),
            text=["End"],
            textposition='top center',
            textfont=dict(color='black', size=18),
            name='End Point'
        ))

        for i in range(len(extra_array) - 1):
            p1 = extra_array[i]
            p2 = extra_array[i + 1]
            direction = p2 - p1
            norm = np.linalg.norm(direction)
            if norm == 0:
                continue
            direction = direction / norm
            arrow_len = 2
            arrow_vec = direction * arrow_len
            arrow_end = p1 + arrow_vec

            fig.add_trace(go.Scatter3d(
                x=[p1[0], arrow_end[0]],
                y=[p1[1], arrow_end[1]],
                z=[p1[2], arrow_end[2]],
                mode='lines',
                line=dict(color='blue', width=4, dash='dot'),
                showlegend=False
            ))

    axis_range = [0, axis_limit] if axis_limit is not None else None

    fig.update_layout(
    scene=dict(
        xaxis=dict(title="x1", showgrid=True,tickfont=dict(size=16),titlefont=dict(size=22)),
        yaxis=dict(title="x2", showgrid=True,tickfont=dict(size=16),titlefont=dict(size=22)),
        zaxis=dict(title="x3", showgrid=True,tickfont=dict(size=16),titlefont=dict(size=22)),
        camera=dict(eye=dict(x=-1.8, y=-1.8, z=1.3)),
        domain=dict(x=[0,1], y=[0,1])  # <-- very important to use full figure
    ),
    width=2000,
    height=1600,
    margin=dict(l=0, r=0, b=0, t=0),  # <-- remove all margins!
    title="3D Polytopes with Critical Region Labels and Trajectory",
    showlegend=False
)

    img_bytes = fig.to_image(format="png", width=width, height=height)

    if save_path is not None:
        # with open(save_path, "wb") as f:
        #     f.write(img_bytes)
        print(f"Saved high-resolution image at {save_path}")

    display(Image(data=img_bytes))

    return CR_index


In [7]:
# Load the CSV file
df = pd.read_csv("../nonzero_values_stacked.csv", header=None)

# Remove rows with any zero elements
df_clean = df[(df != 0).all(axis=1)]


# Display the result
print(df_clean)

FileNotFoundError: [Errno 2] No such file or directory: '../nonzero_values_stacked.csv'

In [8]:
for i in range(df_clean.shape[1]):
    df = pd.read_csv(f"../primal_sols_time_{i+1}.csv", header=None)
    print(df)

NameError: name 'df_clean' is not defined

In [9]:
for i in range(df_clean.shape[1]):
    fixed_values = df_clean.iloc[:, i]
    df = pd.read_csv(f"../primal_sols_time_{i+1}.csv", header=None)

    df_cleaned = df.iloc[1:, :]
    df_numeric = df_cleaned.apply(pd.to_numeric, errors='coerce')

    extra_points = [row[:3].to_numpy(dtype=float) for _, row in df_numeric.iterrows()]

    save_path = f"polytope_plot_time_{i+1}.png"
    CR_in_plot = plot_polytope_from_fixed_values(fixed_values, CRs, extra_points, save_path=save_path)


NameError: name 'df_clean' is not defined